In [ ]:
import pandas as pd
import GladUtil as gd
pd.set_option('display.max_rows', 1000)

대상 = ['장기','자동차','일반','생보']

ym = '202105'

#--곰스 파일정보
path_goms =''
path_ibss =''

in_file = 'e:/글로벌백업(180329)/수수료비교(goms-ibss)/파일정보.xlsx'
df_ibss_nm = pd.read_excel(in_file,sheet_name='파일정보',header=1,usecols="a:e",dtype={'년월':str})
r = df_ibss_nm[(df_ibss_nm['년월']==ym)]
r.loc[:,'gb'] = r['종목'] + r['구분']
r.loc[:,'file'] = r['경로명'] + r['파일명']
r.loc[:,'file'] = r['file'].str.replace('\\','/')
goms_dict = { x:y  for (x,y) in zip(r['gb'],r['file'])}

# -- ibss파일정보 가져오기
in_file = 'E:/글로벌백업(180329)/77.EXCEL/My VBA/수수료작업util_3.0.xlsm'
df_ibss_nm = pd.read_excel(in_file,sheet_name='파일정보',header=1,usecols="a:e",dtype={'년월':str})
r = df_ibss_nm[(df_ibss_nm['년월']==ym)]
r.loc[:,'file'] = r['경로명'] + r['파일명']
r.loc[:,'file'] = r['file'].str.replace('\\','/')
ibss_dict = { x:y  for (x,y) in zip(r['종목'],r['file'])}

## IBSS와 GOMS수수료 산출결과를 비교함(fc)

In [ ]:
gb = '장기'
if not gb in 대상:
    print('작업대상 보종 오류...')

'''곰스 파일'''
if gb=='생보':
    in_file = goms_dict['생보FC']
else:
    in_file = goms_dict['손보FC']
df = pd.read_excel(path_goms+in_file,header=0,dtype={'계약일자':object})
df_g = df[df['구분']==gb].copy()

'''ibss파일'''
# in_file = ibss_file[대상.index(gb)]
in_file = ibss_dict[gb]
df = pd.read_excel(path_ibss+in_file,header=0)
if gb == '장기'  or gb =='생보':
    df_i = df[(df['계약일']>='2021-04-01') | (df['구분']=='초회')].copy()
else:
    df_i = df.copy()

In [ ]:
ibss_cols = ['지점',  '담당','보험회사','보종',      '계약일']
goms_cols = ['지역단','성명','보험사명','상품소분류','계약일자']

if gb == '생보':
    ibss_prm_col = '환산보험료'
    goms_prm_col = '원수사성적'
elif gb == '장기': 
    ibss_prm_col = '수정보험료'
    goms_prm_col = '원수사성적'
else: 
    if gb == '일반':
        ibss_prm_col = '납입보험료'
        ibss_cols = ['지점','담당','보험회사','보험상품','개시일']
    else:
        ibss_prm_col = '보험료'
        ibss_cols = ['지점','담당','보험회사','물건','개시일']
    goms_prm_col = '보험료'
    goms_cols = ['지역단','성명','보험사명','상품명','계약일자']

''' 증권번호 단위로 비교'''
df_i_grp = df_i.groupby(['증권번호'])[['수수료S',ibss_prm_col]].agg(sum)
df_i_grp['지급율_ibss'] = df_i_grp['수수료S'] / df_i_grp[ibss_prm_col] * 100
df_g_grp = df_g.groupby(['증권번호'])[['산출금액',goms_prm_col]].agg(sum)
df_g_grp['지급율_goms'] = df_g_grp['산출금액'] / df_g_grp[goms_prm_col] * 100
df_i_grp = df_i_grp.astype(float)
df_g_grp = df_g_grp.astype(float)

df_ply = pd.merge(df_i_grp,df_g_grp,how='outer',on=['증권번호'],indicator=True).reset_index().astype({"_merge": str}).fillna(0)
df_ply['차이'] = df_ply['수수료S'] - df_ply['산출금액']

# ibss로 부터 데이터가져오기
df_temp = df_i.drop_duplicates(['증권번호'], keep='first')  #증번중복은 첫번째 건만 추출
df_unload = df_ply[df_ply['_merge'] != 'right_only'].join(df_temp.set_index(['증권번호'])[ibss_cols],
                                                          on=(['증권번호']), rsuffix='g', how='left')
# goms로 부터 데이터가져오기
df_temp = df_g.drop_duplicates(['증권번호'], keep='first')
df_temp = df_ply[df_ply['_merge'] == 'right_only'].join(df_temp.set_index(['증권번호'])[goms_cols],
                                                        on=(['증권번호']), rsuffix='g', how='left')
df_temp.columns = df_unload.columns

## 차이발생건만 list up
## left_only = ibss only,  right_only = goms only
df_unload = pd.concat([df_unload,df_temp])
df_unload.rename(columns={'산출금액':'수수료_goms','보험료_x':'보험료_ibss','보험료_y':'보험료_goms'},inplace=True)
df_unload[abs(df_unload['차이']) > 100].sort_values('증권번호')
# ''' to excel from df_unload '''

### IBSS와 GOMS수수료 산출결과를 비교함(OR수수료)
  IBSS는 지사,사원수수료 컬럼있음. GOMS는 지사는 OR수수료파일이 있고 산출금액엔 순수OR금액만 있음

In [ ]:
gb = '자동차'
if not gb in 대상:
    print('작업대상 보종 오류...')
    
'''곰스 파일'''
if gb=='생보':
    in_file = goms_dict['생보OR']
else:
    in_file = goms_dict['손보OR']
df = pd.read_excel(path_goms+in_file,header=0,dtype={'계약일자':object})
df_g = df[df['구분']==gb].copy()
df_g['계약일자'] = ""  #빈 컬럼 삽입

'''ibss파일'''
in_file = ibss_dict[gb]
df = pd.read_excel(path_ibss+in_file,header=0)
if gb == '장기'  or gb =='생보':
    df_i = df[(df['계약일']>='2021-04-01') | (df['구분']=='초회')].copy()
else:
    df_i = df.copy()

In [ ]:
ibss_cols = ['지점',  '담당','보험회사','보종',      '계약일']
goms_cols = ['지역단','성명','보험사명','상품소분류','계약일자']
goms_prm_col = '대상금액'
if gb == '생보':
    ibss_prm_col = '환산보험료'
elif gb == '장기':
    ibss_prm_col = '수정보험료'
else: 
    if gb == '일반':
        ibss_prm_col = '납입보험료'
        ibss_cols = ['지점','담당','보험회사','보험상품','개시일']
    else:
        ibss_prm_col = '보험료'
        ibss_cols = ['지점','담당','보험회사','물건','개시일']
    goms_cols = ['지역단','성명','보험사명','상품명','계약일자']
    
    
df_i = gd.grpConv2(df_i,"지사")
df_i.loc[:,'OR'] = df_i['수수료H'] - df_i['수수료S']    # GOMS방식의 OR수수료 컬럼 생성
df_i.loc[df_i['지사'] !='지사', 'OR'] = 0               # 지사아니면 OR = 0
df_i_grp = df_i.groupby(['증권번호'])[['OR',ibss_prm_col]].agg(sum)
df_g_grp = df_g.groupby(['증권번호'])[['산출금액',goms_prm_col]].agg(sum)

df_ply = pd.merge(df_i_grp,df_g_grp,how='outer',on=['증권번호'],indicator=True).reset_index().astype({"_merge": str}).fillna(0)
df_ply.loc[:, '차이'] = df_ply['OR'] - df_ply['산출금액']

# ibss로 부터 데이터가져오기
df_temp = df_i.drop_duplicates(['증권번호'], keep='first')  #증번중복은 첫번째 건만 추출
df_unload = df_ply[df_ply['_merge'] != 'right_only'].join(df_temp.set_index(['증권번호'])[ibss_cols],
                                                          on=(['증권번호']), rsuffix='g', how='left')

# goms로 부터 데이터가져오기
df_temp = df_g.drop_duplicates(['증권번호'], keep='first')
df_temp = df_ply[df_ply['_merge'] == 'right_only'].join(df_temp.set_index(['증권번호'])[goms_cols],
                                                        on=(['증권번호']), rsuffix='g', how='left')

df_temp.columns = df_unload.columns

## 차이발생건만 list up
## left_only = ibss only,  right_only = goms only
df_unload = pd.concat([df_unload,df_temp])
df_unload.rename(columns={'산출금액':'OR_goms','대상금액':'보험료_goms'},inplace=True)
df_unload[abs(df_unload['차이']) > 100].sort_values('증권번호')
# ''' to excel from df_unload '''

In [ ]:
# df_dif.to_excel('d:/Temp/ss.xlsx')